<a href="https://colab.research.google.com/github/natgluons/GNNs_HGTmodel/blob/main/HGTmodel_GNNs_fraud_nonfraud_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install_deps = False
if install_deps:
    # %pip install https://download.pytorch.org/whl/cu121/torch-2.3.0%2Bcu121-cp312-cp312-win_amd64.whl
    # %pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu
    %pip install torch==2.2.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
    %pip install cudatoolkit
    # %pip install torch_geometric
    %pip install kneed
    %pip install google-cloud-bigquery
    %pip install db-dtypes

In [2]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
import networkx as nx
import torch
import torch.nn.functional as F
# import warnings
# import pandas_gbq
# from sklearn.exceptions import ConvergenceWarning
# from torch_geometric.utils import to_networkx
# from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv
# from torch.optim import Adam
# from torch.nn.functional import log_softmax, nll_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
# from kneed import KneeLocator
from sklearn.manifold import TSNE
# from sklearn.metrics.pairwise import euclidean_distances
# from collections import defaultdict
# from tabulate import tabulate
from google.cloud import bigquery
# from google.oauth2 import service_account

In [3]:
runtime = "local"

In [4]:
if runtime == "colab":
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import auth
    auth.authenticate_user()

In [5]:
def query(project_id):
  query = """
  WITH RECURSIVE
MerchantData AS (
  SELECT
    CAST(biz_org_id AS STRING) AS user_id,
    'merchant' AS type,
    province,
    business_type_desc AS type_desc,
    business_segment,
    business_sub_segment,
    TIMESTAMP_DIFF(CURRENT_DATE(), created_datetime, MONTH) AS account_age_months
  FROM `project-linkaja-dataset.pii.dim_merchant`
),

CustomerData AS (
  SELECT
    CAST(customer_id AS STRING) AS user_id,
    'customer' AS type,
    province,
    trust_level_desc AS type_desc,
    '-' AS business_segment,
    '-' AS business_sub_segment,
    TIMESTAMP_DIFF(CURRENT_DATE(), created_time, MONTH) AS account_age_months
  FROM `project-linkaja-dataset.pii.dim_customer`
),

MerchantHitCounts AS (
  SELECT
    CAST(merchant_id AS STRING) AS user_id,
    COUNT(*) AS hit_count
  FROM `project-self-service-query.fraud_hotlist.dm_fds_rule_hit`
  GROUP BY merchant_id
),

CustomerHitCounts AS (
  SELECT
    CAST(customer_id AS STRING) AS user_id,
    COUNT(*) AS hit_count
  FROM `project-self-service-query.fraud_hotlist.dm_fds_rule_hit`
  GROUP BY customer_id
),

MerchantRiskStatus AS (
  SELECT
    CAST(biz_org_id AS STRING) AS user_id,
    SUM(
      CASE WHEN status_desc = 'blocked' OR status_desc = 'frozen'
      OR biz_org_id IN (SELECT user_id FROM `project-self-service-query.fraud_hotlist.reported_merchant_customer`) THEN 2
      WHEN status_desc = 'suspended' THEN 1
      ELSE 0 END
    ) AS reported_risk
  FROM `project-linkaja-dataset.pii.dim_merchant`
  GROUP BY biz_org_id
),

CustomerRiskStatus AS (
  SELECT
    CAST(customer_id AS STRING) AS user_id,
    SUM(
      CASE WHEN status_desc = 'blocked' OR status_desc = 'frozen'
      OR customer_id IN (SELECT user_id FROM `project-self-service-query.fraud_hotlist.reported_merchant_customer`) THEN 2
      WHEN status_desc = 'suspended' THEN 1
      ELSE 0 END
    ) AS reported_risk
  FROM `project-linkaja-dataset.pii.dim_customer`
  GROUP BY customer_id
),

TransactionData AS (
  SELECT
    COALESCE(d.user_id, c.user_id) AS user_id,
    COALESCE(d.outbound_trx, 0) AS outbound_trx,
    COALESCE(c.inbound_trx, 0) AS inbound_trx
  FROM
    (
        SELECT
            CAST(debit_party_id AS STRING) AS user_id,
            COUNT(orderid) AS outbound_trx
        FROM `project-linkaja-dataset.pii.fact_transaction`
        WHERE partition_initiate_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
          AND debit_party_id IS NOT NULL
          AND debit_party_id IN (SELECT CAST(debit_party_id AS INT64) FROM `project-self-service-query.fraud_hotlist.edges_fraudsampling` WHERE debit_party_id != 'ss_pertamina')
        GROUP BY debit_party_id
    ) d
  FULL OUTER JOIN
    (
        SELECT
            CAST(credit_party_id AS STRING) AS user_id,
            COUNT(orderid) AS inbound_trx
        FROM `project-linkaja-dataset.pii.fact_transaction`
        WHERE partition_initiate_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
          AND credit_party_id IS NOT NULL
          AND credit_party_id IN (SELECT CAST(credit_party_id AS INT64) FROM `project-self-service-query.fraud_hotlist.edges_fraudsampling` WHERE credit_party_id != 'ss_pertamina')
        GROUP BY credit_party_id
    ) c ON d.user_id = c.user_id
),

AggregatedData AS (
  SELECT * FROM (
    SELECT
      COALESCE(MD.user_id, CD.user_id, MHC.user_id, CHC.user_id, MRS.user_id, CRS.user_id) AS user_id,
      COALESCE(MD.type, CD.type) AS type,
      COALESCE(MD.province, CD.province) AS province,
      COALESCE(MD.type_desc, CD.type_desc) AS type_desc,
      COALESCE(MD.business_segment, CD.business_segment) AS business_segment,
      COALESCE(MD.business_sub_segment, CD.business_sub_segment) AS business_sub_segment,
      COALESCE(MD.account_age_months, CD.account_age_months) AS account_age_months,
      TD.inbound_trx,
      TD.outbound_trx,
      COALESCE(MHC.hit_count, CHC.hit_count, 0) AS hit_count,
      COALESCE(MRS.reported_risk, CRS.reported_risk, 0) AS reported_risk
    FROM TransactionData TD
    LEFT JOIN MerchantData MD ON TD.user_id = MD.user_id
    LEFT JOIN CustomerData CD ON TD.user_id = CD.user_id
    LEFT JOIN MerchantHitCounts MHC ON MD.user_id = MHC.user_id
    LEFT JOIN CustomerHitCounts CHC ON CD.user_id = CHC.user_id
    LEFT JOIN MerchantRiskStatus MRS ON MD.user_id = MRS.user_id
    LEFT JOIN CustomerRiskStatus CRS ON CD.user_id = CRS.user_id
    WHERE MD.user_id IS NOT NULL OR CD.user_id IS NOT NULL
    # ORDER BY RAND()
    # LIMIT 17000
  )
    UNION ALL
  SELECT * FROM AggregatedData WHERE FALSE
),

CombinedNodes AS (
  SELECT
    CAST(user_id AS STRING) AS user_id,
    'NULL' AS source_id,
    'NULL' AS target_id,
    NULL AS trans_amount,
    'NULL' AS trans_initiate_time,
    'node' AS data_type,
    CAST(type AS STRING) AS type,
    CAST(province AS STRING) AS province,
    CAST(type_desc AS STRING) AS type_desc,
    CAST(business_segment AS STRING) AS business_segment,
    CAST(business_sub_segment AS STRING) AS business_sub_segment,
    account_age_months,
    outbound_trx,
    inbound_trx,
    hit_count,
    reported_risk
  FROM AggregatedData
),

CombinedEdges AS (
  SELECT
    'NULL' AS user_id,
    CAST(debit_party_id AS STRING) AS source_id,
    CAST(credit_party_id AS STRING) AS target_id,
    trans_amount,
    CAST(trans_initiate_time AS STRING) AS trans_initiate_time,
    'edge' AS data_type,
    'NULL' AS type,
    'NULL' AS province,
    'NULL' AS type_desc,
    'NULL' AS business_segment,
    'NULL' AS business_sub_segment,
    NULL AS account_age_months,
    NULL AS outbound_trx,
    NULL AS inbound_trx,
    NULL AS hit_count,
    NULL AS reported_risk
  FROM `project-linkaja-dataset.pii.fact_transaction`
  WHERE partition_initiate_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
  AND CAST(debit_party_id AS STRING) IN (SELECT user_id FROM AggregatedData WHERE user_id IS NOT NULL)
  AND CAST(credit_party_id AS STRING) IN (SELECT user_id FROM AggregatedData WHERE user_id IS NOT NULL)
)

SELECT * FROM CombinedNodes
UNION ALL
SELECT * FROM CombinedEdges
  """

  if runtime == "colab":
    df = pd.read_gbq(query, project_id=project_id, dialect='standard')
    # query_job = client.query(query)
    # df = query_job.to_dataframe()
  else:
    client = bigquery.Client(project=project_id)
    query_job = client.query(query)
    df = query_job.to_dataframe()
    
  return df


In [6]:
def preprocess(df):
	# Separating nodes and edges
	node = df[['user_id', 'type', 'province', 'type_desc', 'business_segment', 'business_sub_segment', 'account_age_months', 'outbound_trx', 'inbound_trx', 'hit_count', 'reported_risk', 'data_type']]
	nodes = node[node['data_type'] == 'node'].drop('data_type', axis=1)

	edges = df[df['data_type'] == 'edge'].drop(['data_type'] + list(nodes.columns.difference(['source_id', 'target_id'])), axis=1)

	nodes['user_id'] = nodes['user_id'].astype('int64')
	nodes['account_age_months'] = nodes['account_age_months'].astype('int64')
	nodes['outbound_trx'] = nodes['outbound_trx'].astype('int64')
	nodes['inbound_trx'] = nodes['inbound_trx'].astype('int64')
	nodes['hit_count'] = nodes['hit_count'].astype('int64')
	nodes['reported_risk'] = nodes['reported_risk'].astype('int64')

	edges['source_id'] = edges['source_id'].astype('int64')
	edges['target_id'] = edges['target_id'].astype('int64')
	edges['trans_amount'] = edges['trans_amount'].astype('int64')

	#  Create a unified index mapping
	all_ids = pd.concat([nodes['user_id'], edges['source_id'], edges['target_id']]).unique()
	id_to_index = {id: idx for idx, id in enumerate(all_ids)}

	# Make a new column to replace original IDs with indices
	nodes['userid'] = nodes['user_id'].map(id_to_index)
	edges['sourceid'] = edges['source_id'].map(id_to_index)
	edges['targetid'] = edges['target_id'].map(id_to_index)

	return nodes, edges

In [7]:
### RGCN Model Training & Clustering ###

class RGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_relations):
        super(RGCN, self).__init__()
        self.conv1 = RGCNConv(in_channels, hidden_channels, num_relations=num_relations)
        self.conv2 = RGCNConv(hidden_channels, out_channels, num_relations=num_relations)

    def forward(self, x, edge_index, edge_type):
        x = F.relu(self.conv1(x, edge_index, edge_type=edge_type))
        x = F.relu(self.conv2(x, edge_index, edge_type=edge_type))
        return x

In [8]:
def encode(nodes, edges):
    ### Preprocessing for RGCN ###

    # Encode categorical attributes for nodes
    encoder = LabelEncoder()
    for col in ['type', 'province', 'type_desc', 'business_segment', 'business_sub_segment']:
        nodes[col + '_code'] = encoder.fit_transform(nodes[col])
    node_features = torch.tensor(nodes[['type_code', 'province_code', 'type_desc_code', 'business_segment_code', 'business_sub_segment_code', 'account_age_months', 'inbound_trx', 'outbound_trx']].values, dtype=torch.float)
    # node_labels = torch.tensor(nodes['reported_risk'].values, dtype=torch.long)
    # labeled_mask = node_labels != 0

    edge_index = torch.tensor(edges[['sourceid', 'targetid']].values.T, dtype=torch.long)
    edge_index[edge_index < 0] = 0
    # edge_attr = torch.tensor(edges[['trans_amount']].values, dtype=torch.float)
    edge_type = torch.zeros(edges.shape[0], dtype=torch.long)

    return node_features, edge_index, edge_type

In [9]:
def load_model(model_path):
	model = RGCN(in_channels=8, hidden_channels=16, out_channels=3, num_relations=2)  # Adjust in_channels according to your input feature size
	model.load_state_dict(torch.load(model_path))
	return model

In [10]:
def predict(model, node_features, edge_index, edge_type):
	###  Load Trained Model and Use It for Inference ###
	model.eval()

	# Generate embeddings
	with torch.no_grad():
		embeddings = model(node_features, edge_index, edge_type)

	# Use t-SNE to project embeddings to two dimensions for visualization
	tsne = TSNE(n_components=2, perplexity=30, random_state=42)
	embeddings_2d = tsne.fit_transform(embeddings)

	# Apply K-Means to the t-SNE 2D embeddings
	kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
	clusters = kmeans.fit_predict(embeddings_2d)

	return clusters

In [11]:
def make_graph(edge_index):
    # Create a NetworkX graph from the edge_index tensor
    G = nx.DiGraph()  # Use DiGraph to calculate in-degree and out-degree separately
    edge_index_np = edge_index.cpu().numpy()
    for i in range(edge_index_np.shape[1]):
        source = edge_index_np[0, i].item()
        target = edge_index_np[1, i].item()
        G.add_edge(source, target)

    # Calculate network metrics
    degree_centrality = nx.degree_centrality(G)
    in_degree_centrality = nx.in_degree_centrality(G)
    out_degree_centrality = nx.out_degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)

    # Convert centrality measures to DataFrame
    centrality_df = pd.DataFrame({
        'user_id': list(degree_centrality.keys()),
        'degree_centrality': list(degree_centrality.values()),
        'in_degree_centrality': list(in_degree_centrality.values()),
        'out_degree_centrality': list(out_degree_centrality.values()),
        'betweenness_centrality': list(betweenness_centrality.values())
    })

    return centrality_df

In [12]:
def identify_syndicate(nodes, centrality_df):
    # Normalize centrality measures to range between 0 and 1
    centrality_df['normalized_in_degree'] = centrality_df['in_degree_centrality'] / centrality_df['in_degree_centrality'].max()
    centrality_df['normalized_out_degree'] = centrality_df['out_degree_centrality'] / centrality_df['out_degree_centrality'].max()

    # Define weights for each centrality measure
    weights = {
        'in_degree': 0.5,
        'out_degree': 0.5
    }

    # Calculate the weighted syndicate score
    centrality_df['syndicate_score'] = (weights['in_degree'] * centrality_df['normalized_in_degree'] +
                                        weights['out_degree'] * centrality_df['normalized_out_degree'])

    # Scale the score to be between 0 and 1
    max_score = centrality_df['syndicate_score'].max()
    if max_score > 0:  # Avoid division by zero
        centrality_df['syndicate_score'] /= max_score

    # Assuming 'nodes' is your main DataFrame containing user details and is globally accessible or passed as a parameter
    nodes = pd.merge(nodes, centrality_df[['user_id', 'syndicate_score']], on='user_id', how='left')

    return nodes

In [13]:
# Compliance risk mapping for multiple entries
def map_compliance_risk(province, type_desc, business_segment, business_sub_segment):
    # Define the compliance risk mapping
    high_risk_values = (
        'PT', 'Yayasan', 'Luar negeri', 'Jawa Timur', 'DKI Jakarta', 'Jawa Tengah', 'Jawa Barat',
        'Non Profit', 'Corporate', 'e-commerce', 'Disbursement', 'Bank', 'E-Commerce/online',
        'ECommerce/online', 'E-Commerce online', 'E-Commerceonline', 'ECommerceonline', 'Financial Service'
    )
    medium_risk_values = (
        'CV', 'Koperasi', 'Perorangan', 'Jawa Barat', 'Jawa Timur', 'Sumatera Selatan',
        'Sumatera Utara', 'Sumatera Selatan', 'Sumatera Utara', 'Aceh', 'Agent', 'retail', 'F&B',
        'Utility', 'Entertainment', 'Internal and operational', 'Physical Retail', 'Transportation',
        'Ticketing', 'Telco Product', 'Biller', 'Agent Non-Telco', 'Modern Channel', 'agentownshop',
        'Modern Agent Telkomsel', 'Content', 'Games'
    )

    # Check if the input values belong to high or medium risk category
    if province in high_risk_values or type_desc in high_risk_values or business_segment in high_risk_values or business_sub_segment in high_risk_values:
        return 'high'
    elif province in medium_risk_values or type_desc in medium_risk_values or business_segment in medium_risk_values or business_sub_segment in medium_risk_values:
        return 'medium'
    else:
        return 'low'

In [14]:
# Heuristic thresholds and mappings
hit_count_thresholds = {'high': 332, 'medium': 3}

# Function to calculate individual node risk scores
def calculate_node_risk_score(row):
    # Primary factor: Syndicate Score directly influences the base score
    # Assuming syndicate_score is normalized between 0 and 1
    base_score = row['syndicate_score']

    # Secondary adjustments
    # Hit count risk - smaller impact
    if row['hit_count'] > hit_count_thresholds['high']:
        base_score += 0.1  # Small adjustment for extreme cases
    elif row['hit_count'] > hit_count_thresholds['medium']:
        base_score += 0.05  # Even smaller adjustment for medium cases

    # Compliance risk - using a mapped function to get risk adjustment
    compliance_adjustment = map_compliance_risk(row['province'], row['type_desc'], row['business_segment'], row['business_sub_segment'])
    if compliance_adjustment == 'high':
        base_score += 0.1  # Compliance issues can adjust the score slightly
    elif compliance_adjustment == 'medium':
        base_score += 0.05

    # Cap the score at 1.0 since the scale is 0 to 1
    final_score = min(base_score, 1.0)
    return final_score

In [15]:
def calculate_risk_scores(nodes):
	# Calculate risk scores for each node
	nodes['risk_index'] = nodes.apply(calculate_node_risk_score, axis=1)

	# Aggregate risk scores by RGCN_cluster to get the average risk score for each cluster
	cluster_risk_scores = nodes.groupby('RGCN_cluster')['risk_index'].mean().reset_index()
	cluster_risk_scores.rename(columns={'risk_index': 'cluster_risk_score'}, inplace=True)

	# Calculate the thresholds for 'low' and 'high' risk levels based on quantiles
	low_threshold = cluster_risk_scores['cluster_risk_score'].quantile(0.33)
	high_threshold = cluster_risk_scores['cluster_risk_score'].quantile(0.67)

	# Define function to assign risk levels based on quantile thresholds
	def assign_risk_level(final_score):
		if final_score >= high_threshold:
			return 'high'
		elif final_score > low_threshold:
			return 'medium'
		else:
			return 'low'

	# Assign risk levels based on quantiles
	cluster_risk_scores['cluster_risk_level'] = cluster_risk_scores['cluster_risk_score'].apply(assign_risk_level)

	nodes = nodes.merge(cluster_risk_scores, on='RGCN_cluster', how='left')

	return nodes, cluster_risk_scores

In [16]:
# Set adjustment factors
degree_adjustment_factor = 0.05
betweenness_adjustment_factor = 0.10
def calculate_risk_index(cluster_risk_score, degree_centrality, betweenness_centrality):
    # Adjust the risk index for each merchant node based on centrality measures
    adjusted_risk_index = cluster_risk_score + \
        (degree_centrality * degree_adjustment_factor) + \
        (betweenness_centrality * betweenness_adjustment_factor)

    # Normalize the adjusted risk index using Min-Max scaling
    min_risk_index = adjusted_risk_index.min()
    max_risk_index = adjusted_risk_index.max()
    risk_index = (adjusted_risk_index - min_risk_index) / (max_risk_index - min_risk_index)

    # Calculate quantile thresholds for 'low' and 'high' risk levels
    low_quantile = risk_index.quantile(0.33)
    high_quantile = risk_index.quantile(0.67)

    # Define function to assign merchant risk levels based on quantile thresholds
    def assign_risk_level(score):
        if score >= high_quantile:
            return 'high'
        elif score > low_quantile:
            return 'medium'
        else:
            return 'low'

    # Assign risk levels based on quantiles
    risk_level = risk_index.apply(assign_risk_level)

    return adjusted_risk_index, risk_index, risk_level

In [17]:
def load_data_and_process(project_id):
    # Assuming query is a function that retrieves your dataset
    df = query(project_id)

    # Assuming preprocess is a function that prepares your data
    nodes, edges = preprocess(df)

    # Assuming encode is a function that extracts features and encodes them for the model
    node_features, edge_index, edge_type = encode(nodes, edges)

    return nodes, edges, node_features, edge_index, edge_type

def workflow(model_path, nodes, node_features, edge_index, edge_type):
    # Load the predictive model
    model = load_model(model_path)

    # Predict cluster memberships
    nodes['RGCN_cluster'] = predict(model, node_features, edge_index, edge_type)

    # Generate graph and calculate centrality metrics
    centrality_df = make_graph(edge_index)
    nodes = nodes.merge(centrality_df, on='user_id', how='left')
    nodes['degree_centrality'].fillna(0, inplace=True)
    nodes['betweenness_centrality'].fillna(0, inplace=True)

    # Identify syndicate nodes using the centrality data
    nodes = identify_syndicate(nodes, centrality_df)

    # Calculate risk scores incorporating new syndicate scores
    nodes, cluster_risk_scores = calculate_risk_scores(nodes)

    # Aggregate users within each cluster
    clustered_users = nodes.groupby('RGCN_cluster')['user_id'].apply(list).reset_index()
    clustered_users['user_id'] = clustered_users['user_id'].apply(lambda x: ', '.join(map(str, x)))

    # Assuming calculate_risk_index is a function to adjust risk indices based on new metrics
    adjusted_risk_index, risk_index, risk_level = calculate_risk_index(
        nodes['cluster_risk_score'], nodes['degree_centrality'], nodes['betweenness_centrality'])

    nodes['adjusted_risk_index'] = adjusted_risk_index
    nodes['risk_index'] = risk_index
    nodes['risk_level'] = risk_level

    return nodes, cluster_risk_scores, clustered_users

In [18]:
def modify_nodes_df(nodes):
    # Create new columns by aggregating other columns into dictionaries
    nodes['attributes'] = nodes[['province', 'type_desc', 'business_segment',
                                 'business_sub_segment', 'account_age_months', 'outbound_trx',
                                 'inbound_trx', 'hit_count', 'reported_risk']].apply(lambda x: x.to_dict(), axis=1)
    nodes['network_metrics'] = nodes[['degree_centrality',
                                      'in_degree_centrality', 'out_degree_centrality',
                                      'betweenness_centrality']].apply(lambda x: x.to_dict(), axis=1)

    # Drop specified columns
    nodes.drop(columns=['userid', 'type_code', 'province_code', 'type_desc_code',
                        'business_segment_code', 'business_sub_segment_code'], inplace=True)

    # Prepare final DataFrame
    final_result = nodes[['user_id', 'type', 'attributes', 'network_metrics', 'syndicate_score', 'RGCN_cluster', 'cluster_risk_level', 'risk_level', 'cluster_risk_score', 'risk_index', 'adjusted_risk_index']]
    final_result = final_result.sort_values(by='syndicate_score', ascending=False)

    return final_result

In [20]:
# Main execution
project_id = "project-linkaja-dataset"
model_path = "./models/RGCN_model.pth"
if runtime == "colab":
    model_path = "/content/drive/My Drive/RGCN_model.pth"

# Load data and prepare it for the model
nodes, edges, node_features, edge_index, edge_type = load_data_and_process(project_id)

# Run the model and calculate risk scores
nodes, cluster_risk_scores, clustered_users = workflow(
    model_path, nodes, node_features, edge_index, edge_type)

final_result = modify_nodes_df(nodes)
final_result

C:\Users\KristyNatashaYohanes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,user_id,type,attributes,network_metrics,syndicate_score,RGCN_cluster,cluster_risk_level,risk_level,cluster_risk_score,risk_index,adjusted_risk_index
0,202000000091157829,customer,"{'province': 'Jawa Tengah', 'type_desc': 'full...","{'degree_centrality': 0.0, 'in_degree_centrali...",NaN,2,low,low,NaN,NaN,NaN
1,202000000087309664,customer,"{'province': 'Jawa Tengah', 'type_desc': 'full...","{'degree_centrality': 0.0, 'in_degree_centrali...",NaN,1,low,low,NaN,NaN,NaN
2,202000000095849332,customer,"{'province': 'Jawa Tengah', 'type_desc': 'full...","{'degree_centrality': 0.0, 'in_degree_centrali...",NaN,2,low,low,NaN,NaN,NaN
3,202100000009571443,customer,"{'province': 'Jawa Timur', 'type_desc': 'full ...","{'degree_centrality': 0.0, 'in_degree_centrali...",NaN,2,low,low,NaN,NaN,NaN
4,202100000007495194,customer,"{'province': 'Jawa Barat', 'type_desc': 'full ...","{'degree_centrality': 0.0, 'in_degree_centrali...",NaN,2,low,low,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1453,202000000039824612,customer,"{'province': 'Jawa Timur', 'type_desc': 'full ...","{'degree_centrality': 0.0, 'in_degree_centrali...",NaN,1,low,low,NaN,NaN,NaN
1454,202000000097612804,customer,"{'province': 'Jawa Barat', 'type_desc': 'full ...","{'degree_centrality': 0.0, 'in_degree_centrali...",NaN,1,low,low,NaN,NaN,NaN
1455,202000000062562373,customer,"{'province': 'Jawa Barat', 'type_desc': 'full ...","{'degree_centrality': 0.0, 'in_degree_centrali...",NaN,1,low,low,NaN,NaN,NaN
1456,202100000009332538,customer,"{'province': 'Jawa Barat', 'type_desc': 'full ...","{'degree_centrality': 0.0, 'in_degree_centrali...",NaN,0,low,low,NaN,NaN,NaN


In [ ]:
numeric_stats = final_result.describe()
numeric_stats

In [ ]:
cluster_risk_scores

In [ ]:
clustered_users